### Preparação do ambiente

In [1]:
%pip install -q scikit-learn pandas matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importação de bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import time
import psutil
import os

### Carregamento dos dados

In [3]:
# URLs dos dados
url_train = "https://raw.githubusercontent.com/pcbrom/perceptron-mlp-cnn/refs/heads/main/data/train.csv"
url_test = "https://raw.githubusercontent.com/pcbrom/perceptron-mlp-cnn/refs/heads/main/data/test.csv"
url_validation = "https://raw.githubusercontent.com/pcbrom/perceptron-mlp-cnn/refs/heads/main/data/validation.csv"

# Leitura dos dados e seleção das 9 primeiras colunas
df_train = pd.read_csv(url_train).iloc[:, :9]
df_test = pd.read_csv(url_test).iloc[:, :9]
df_validation = pd.read_csv(url_validation).iloc[:, :9]

### Pré-processamento

In [4]:
# Imputação dos valores ausentes com a mediana
imputer = SimpleImputer(strategy="median")

# Padronização dos dados
scaler = StandardScaler()

# Separação entre features (X) e alvo (y) para cada dataset
X_train = df_train.drop("Outcome", axis=1)
y_train = df_train["Outcome"]

X_test = df_test.drop("Outcome", axis=1)
y_test = df_test["Outcome"]

X_val = df_validation.drop("Outcome", axis=1)
y_val = df_validation["Outcome"]

# Imputação de dados ausentes e padronização
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
X_val_imputed = imputer.transform(X_val)

# Padronização dos dados
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)
X_val_scaled = scaler.transform(X_val_imputed)

### MLP
1. Forward pass: Propagação dos dados pela rede (entradas → camadas ocultas → saída).

2. Backpropagation: Cálculo do gradiente e atualização dos pesos (algoritmo de retropropagação).

3. Treinamento: Usamos o gradiente descendente para otimizar os pesos da rede.

In [5]:
# MultiLayer Perceptron
class MLP:
    def __init__(self, input_dim, hidden_dim, output_dim, learning_rate=0.01, random_state=42):
        # Inicializa as camadas e os pesos
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        
        # Define seed para reprodutibilidade
        np.random.seed(random_state)

        # Inicialização Xavier/Glorot para melhor convergência
        self.W1 = np.random.randn(self.input_dim, self.hidden_dim) * np.sqrt(2.0 / self.input_dim)
        self.b1 = np.zeros((1, self.hidden_dim))
        self.W2 = np.random.randn(self.hidden_dim, self.output_dim) * np.sqrt(2.0 / self.hidden_dim)
        self.b2 = np.zeros((1, self.output_dim))
        
        # Histórico de loss para acompanhar convergência
        self.loss_history = []
    
    def sigmoid(self, z):
        # Função de ativação sigmoide com clipping para evitar overflow
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        # Derivada da função sigmoide
        return z * (1 - z)
    
    def forward(self, X):
        # Passagem para frente
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2
    
    def backward(self, X, y):
        # Backpropagation
        m = X.shape[0]
        
        # Reshape y para garantir dimensões corretas
        if y.ndim == 1:
            y = y.reshape(-1, 1)
            
        self.output_error = self.a2 - y
        self.dZ2 = self.output_error * self.sigmoid_derivative(self.a2)
        self.dW2 = np.dot(self.a1.T, self.dZ2) / m
        self.db2 = np.sum(self.dZ2, axis=0, keepdims=True) / m

        self.dZ1 = np.dot(self.dZ2, self.W2.T) * self.sigmoid_derivative(self.a1)
        self.dW1 = np.dot(X.T, self.dZ1) / m
        self.db1 = np.sum(self.dZ1, axis=0, keepdims=True) / m
        
    def update_weights(self):
        # Atualiza os pesos e vieses
        self.W1 -= self.learning_rate * self.dW1
        self.b1 -= self.learning_rate * self.db1
        self.W2 -= self.learning_rate * self.dW2
        self.b2 -= self.learning_rate * self.db2
    
    def compute_loss(self, y_true, y_pred):
        # Binary cross-entropy loss (mais apropriado para classificação binária)
        if y_true.ndim == 1:
            y_true = y_true.reshape(-1, 1)
        
        # Clipping para evitar log(0)
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    
    def train(self, X, y, epochs=1000, verbose=True):
        # Treinamento da rede neural
        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y)
            self.update_weights()
            
            # Calcula e armazena a loss
            loss = self.compute_loss(y, self.a2)
            self.loss_history.append(loss)
            
            if verbose and epoch % 100 == 0:
                print(f"Epoch {epoch}/{epochs}, Loss: {loss:.4f}")
    
    def predict(self, X):
        # Predições
        return self.forward(X)
    
    def predict_classes(self, X, threshold=0.5):
        # Predições de classe
        probabilities = self.predict(X)
        return (probabilities >= threshold).astype(int)

### Treinamento e avaliação do MLP

In [6]:
# Inicializando e treinando o MLP
mlp = MLP(input_dim=X_train_scaled.shape[1], hidden_dim=8, output_dim=1, learning_rate=0.01)

# Treinamento (usando 100000 épocas) - convertendo Series para array NumPy
mlp.train(X_train_scaled, y_train.values, epochs=100000)

# Previsões com o MLP
y_pred_mlp_proba = mlp.predict(X_test_scaled)
y_pred_mlp = mlp.predict_classes(X_test_scaled)

# Flatten para garantir dimensões corretas
y_pred_mlp = y_pred_mlp.flatten()

# Avaliação do MLP
print("\nAvaliação do MLP:")
print("Acurácia:", accuracy_score(y_val, y_pred_mlp))
print("Precisão:", precision_score(y_val, y_pred_mlp))
print("Revocação (Recall):", recall_score(y_val, y_pred_mlp))
print("F1-score:", f1_score(y_val, y_pred_mlp))

Epoch 0/100000, Loss: 1.0681
Epoch 100/100000, Loss: 0.9533
Epoch 200/100000, Loss: 0.8534
Epoch 300/100000, Loss: 0.7739
Epoch 400/100000, Loss: 0.7151
Epoch 500/100000, Loss: 0.6737
Epoch 600/100000, Loss: 0.6453
Epoch 700/100000, Loss: 0.6257
Epoch 800/100000, Loss: 0.6121
Epoch 900/100000, Loss: 0.6024
Epoch 1000/100000, Loss: 0.5954
Epoch 1100/100000, Loss: 0.5901
Epoch 1200/100000, Loss: 0.5860
Epoch 1300/100000, Loss: 0.5827
Epoch 1400/100000, Loss: 0.5800
Epoch 1500/100000, Loss: 0.5778
Epoch 1600/100000, Loss: 0.5758
Epoch 1700/100000, Loss: 0.5740
Epoch 1800/100000, Loss: 0.5724
Epoch 1900/100000, Loss: 0.5710
Epoch 2000/100000, Loss: 0.5696
Epoch 2100/100000, Loss: 0.5683
Epoch 2200/100000, Loss: 0.5670
Epoch 2300/100000, Loss: 0.5658
Epoch 2400/100000, Loss: 0.5646
Epoch 2500/100000, Loss: 0.5635
Epoch 2600/100000, Loss: 0.5623
Epoch 2700/100000, Loss: 0.5612
Epoch 2800/100000, Loss: 0.5601
Epoch 2900/100000, Loss: 0.5591
Epoch 3000/100000, Loss: 0.5580
Epoch 3100/100000, L

### Comparação RL e MLP

In [7]:
# Primeira parte - Regressão Logística
start_time = time.time()
logreg = LogisticRegression(max_iter=100000, random_state=42)
logreg.fit(X_train_scaled, y_train)
y_pred_logreg = logreg.predict(X_val_scaled)
logreg_time = time.time() - start_time

# Avaliação da regressão logística
logreg_accuracy = accuracy_score(y_val, y_pred_logreg)
logreg_precision = precision_score(y_val, y_pred_logreg, zero_division=0)
logreg_recall = recall_score(y_val, y_pred_logreg, zero_division=0)
logreg_f1 = f1_score(y_val, y_pred_logreg, zero_division=0)

# Segunda parte - MLP
start_time = time.time()
mlp_comparison = MLP(input_dim=X_train_scaled.shape[1], hidden_dim=8, output_dim=1, 
                    learning_rate=0.01, random_state=42)
mlp_comparison.train(X_train_scaled, y_train.values, epochs=100000, verbose=False)
y_pred_mlp_comparison = mlp_comparison.predict_classes(X_val_scaled).flatten()
mlp_time = time.time() - start_time

# Avaliação do MLP
mlp_accuracy = accuracy_score(y_val, y_pred_mlp_comparison)
mlp_precision = precision_score(y_val, y_pred_mlp_comparison, zero_division=0)
mlp_recall = recall_score(y_val, y_pred_mlp_comparison, zero_division=0)
mlp_f1 = f1_score(y_val, y_pred_mlp_comparison, zero_division=0)

# Exibindo os resultados
comparison_df = pd.DataFrame({
    'Model': ['Logistic Regression', 'MLP'],
    'Accuracy': [logreg_accuracy, mlp_accuracy],
    'Precision': [logreg_precision, mlp_precision],
    'Recall': [logreg_recall, mlp_recall],
    'F1-Score': [logreg_f1, mlp_f1],
    'Time (s)': [logreg_time, mlp_time]
})

print("\nComparação dos Modelos:")
print(comparison_df)


Comparação dos Modelos:
                 Model  Accuracy  Precision  Recall  F1-Score   Time (s)
0  Logistic Regression  0.813559   0.736842    0.70  0.717949   0.018258
1                  MLP  0.779661   0.684211    0.65  0.666667  10.089914


### Análise da complexidade

In [8]:
def measure_complexity(model_func, X, y, model_name):
    """
    Função para medir o uso de memória e tempo de treinamento
    """
    start_time = time.time()
    
    # Memória inicial (MB)
    initial_memory = psutil.Process(os.getpid()).memory_info().rss / 1024**2
    
    # Executa o treinamento
    model = model_func(X, y)
    
    # Tempo de execução e memória final (MB)
    end_time = time.time()
    final_memory = psutil.Process(os.getpid()).memory_info().rss / 1024**2
    
    execution_time = end_time - start_time
    memory_usage = final_memory - initial_memory
    
    print(f"{model_name} - Time: {execution_time:.4f}s, Memory: {memory_usage:.4f}MB")
    
    return execution_time, memory_usage, model

def train_logistic_regression(X, y):
    """Função para treinar regressão logística"""
    model = LogisticRegression(max_iter=100000, random_state=42)
    model.fit(X, y)
    return model

def train_mlp(X, y):
    """Função para treinar MLP"""
    model = MLP(input_dim=X.shape[1], hidden_dim=8, output_dim=1, 
               learning_rate=0.01, random_state=42)
    model.train(X, y, epochs=100000, verbose=False)
    return model

# Comparando a complexidade computacional dos modelos
print("Análise de Complexidade Computacional:")
print("=" * 50)

logreg_time, logreg_memory, logreg_model = measure_complexity(
    train_logistic_regression, X_train_scaled, y_train, "Logistic Regression"
)

mlp_time, mlp_memory, mlp_model = measure_complexity(
    train_mlp, X_train_scaled, y_train.values, "MLP"  # Convertendo para array NumPy
)

# Resumo da comparação
print("\nResumo da Comparação:")
print(f"Speedup (MLP vs LogReg): {mlp_time/logreg_time:.2f}x mais lento")
print(f"Memory overhead (MLP vs LogReg): {mlp_memory/max(logreg_memory, 0.001):.2f}x mais memória")

Análise de Complexidade Computacional:
Logistic Regression - Time: 0.0046s, Memory: 0.0000MB
MLP - Time: 9.6432s, Memory: 3.6016MB

Resumo da Comparação:
Speedup (MLP vs LogReg): 2115.52x mais lento
Memory overhead (MLP vs LogReg): 3601.56x mais memória
